In [ ]:
!pip install matplotlib

In [ ]:
import syft as sy
%matplotlib inline
duet = sy.join_duet(loopback=True)

In [ ]:
class SyNet(sy.Module):
    def __init__(self, torch_ref):
        super(SyNet, self).__init__(torch_ref=torch_ref)
        
        self.lin =self.torch_ref.nn.Linear(6,10)
    def forward(self, x):
        x = self.lin(x)
        return x

In [ ]:
import torch
model1 = SyNet(torch)
model1_ptr = model1.send(duet)

In [ ]:
opt1 = duet.torch.optim.Adam(params=model1_ptr.parameters(),lr=0.01)

In [ ]:
duet.store.pandas

In [ ]:
data_pointer = duet.store[0]

In [ ]:
import numpy as np

In [ ]:

model2 = torch.nn.Linear(10,6)
opt2 = torch.optim.Adam(params=model2.parameters(), lr=0.01)

target = torch.tensor([[16.03,16.04,16.05,16.06,16.07,16.08],[16.09,16.1,16.11,16.11,18.22,18.25],
                       [18.26,18.27,18.28,18.29,18.3,19.26],[19.32,19.33,19.43,19.51,19.62,19.73],
                       [19.81,19.95,20.6,20.61,20.62,20.43],[20.44,21.23,21.41,21.45,21.53,21.64],
                       [21.67,21.72,21.75,21.84,21.86,21.88],[21.92,23.77,24.85,25.79,28.07,32.47],
                       [35.49,35.85,40.84,44.59,44.77,46.21],[50.8,52.12,53,58.88,60.81,63.25]])

target.shape
#target.type

In [ ]:
import matplotlib.pyplot as plt
#epoch = 200
losses = []
predictions = []
for iter in range(250):
    opt1.zero_grad()
    opt2.zero_grad()
    
    activation_ptr = model1_ptr(data_pointer)
    activation = activation_ptr.clone().get(request_block=True)
    
    pred = model2(activation)
    
    #target = target.detach().numpy()
    loss = ((pred - target)**2).sum()
   
    loss.backward()
    
    grad_ptr = activation.grad.clone().send(duet)
    activation_ptr.backward(grad_ptr)
    
    opt1.step()
    opt2.step()
    losses.append(loss.tolist())
    predictions.append(pred.tolist())
    
    print("losses: ",losses)
    print('Predictions =',pred)


In [ ]:
import matplotlib.pyplot as plt
#%matplotlib inline
#pred = pred.detach().numpy()
#pred = pred.numpy()
#loss = loss.detach().numpy()
# print("lost",losses)
# print("list",losses[0].tolist())
# print("pred", predictions[0].tolist())
# print("pred", predictions[0].ravel().tolist())
#model2 = model2.detach().numpy()
#target = target.detach().numpy()
plt.title('Diameter of Bulb,AEA Hydroponics')
plt.plot(losses, color='green')
plt.ylabel("Loss")
plt.xlabel("epoch")
plt.show()